In [1]:
import joblib
from pytorch_lightning.utilities.seed import seed_everything
from collie.metrics import auc, evaluate_in_batches, mapk, mrr
from collie.model import CollieTrainer, MatrixFactorizationModel
from collie.movielens import get_recommendation_visualizations

c:\Users\defaultuser0.LAPTOP-POTNR7PE\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
import gc
import torch

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name())
print(torch.cuda.memory_allocated())

True
0
NVIDIA GeForce GTX 1650
0


In [4]:
import os
parent_directory = r"d:\\When-Hybrid-meets-the-Prompt"

os.chdir(parent_directory)
os.getcwd()

'd:\\When-Hybrid-meets-the-Prompt'

In [5]:
train_interactions = joblib.load("Data/train_interactions_10K.pkl")
val_interactions = joblib.load("Data/val_interactions_10K.pkl")

In [6]:
# train_loader = ApproximateNegativeSamplingInteractionsDataLoader(train_interactions, batch_size=1024, shuffle=True)
# val_loader = ApproximateNegativeSamplingInteractionsDataLoader(val_interactions, batch_size=1024, shuffle=False)

In [7]:
seed_everything(42)

c:\Users\defaultuser0.LAPTOP-POTNR7PE\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\utilities\seed.py:47: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
Global seed set to 42


42

In [8]:
model = MatrixFactorizationModel(
    train=train_interactions,
    val=val_interactions,
    embedding_dim=10,
    lr=1e-2,
)

In [9]:
torch.cuda.empty_cache()
gc.collect()

10

In [10]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [12]:
trainer = CollieTrainer(model, max_epochs=1, deterministic=True, benchmark=True, accelerator = "gpu", 
                        auto_scale_batch_size = True,callbacks=[EarlyStopping(monitor='val_loss_epoch')])

trainer.fit(model)

Detected GPU. Setting ``gpus`` to 1.


c:\Users\defaultuser0.LAPTOP-POTNR7PE\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:152: UserWarning: You passed `deterministic=True` and `benchmark=True`. Note that PyTorch ignores torch.backends.cudnn.deterministic=True when torch.backends.cudnn.benchmark=True.
  rank_zero_warn(
c:\Users\defaultuser0.LAPTOP-POTNR7PE\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: d:\When-Hybrid-meets-the-Prompt\lightning_logs


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.68 GiB (GPU 0; 4.00 GiB total capacity; 583.81 MiB already allocated; 2.66 GiB free; 602.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF